<font size="8">HLA-A:02*01 Cross validation with Leave One Out (LOO)</font>

Idea is to create 5 clusters using https://services.healthtech.dtu.dk/services/GibbsCluster-2.0/, run the experiment 5 times with each time leave a different cluster out as test set

In [3]:
import pandas as pd

hla_a2 = pd.read_csv("y:/data/hla_a_02_01.csv")
missing_peptide = pd.read_csv("Y:\data\missing_ids\BA_pMHCI_human_quantitative_HLA_A2_missing_graph.csv")
merged_df = pd.merge(hla_a2, missing_peptide, on='ID', how='inner')

# Filter the rows in df1 where the ids match with df2
filtered_hla_a2 = hla_a2[~hla_a2['ID'].isin(merged_df['ID'])]

In [7]:
print(len(filtered_hla_a2))
print(filtered_hla_a2.columns)
filtered_hla_a2.to_csv("y:/data/hla_a_02_01_structure_present.csv", index=False)

8225
Index(['ID', 'allele', 'peptide', 'measurement_value',
       'measurement_inequality', 'measurement_type', 'measurement_kind',
       'measurement_source', 'original_allele', 'db2_folder', 'cluster',
       'anchor_0', 'anchor_1', 'cluster_set_10', 'allele_clustering',
       'binder'],
      dtype='object')


In [9]:
peptide_filtered = filtered_hla_a2.peptide
peptide_filtered.to_csv("y:/data/hla_a_02_01_peptide_structure_present.csv", index=False)

Peptides have now been clustered, perform additional statistics.

Remember: some sequences are double, so when adding ID you should be back a the old amount of ID

Create a dataframe, containing 4 columns; Cluster; ID; Sequence; Binder

In [35]:
gibbs_report = pd.read_csv("Y:\data\hla_a_02_01_peptide_5_clusters.csv", header = None)
print(len(gibbs_report))
cluster_label = 0
cluster_labels = []

# Iterate through each row
for index, row in gibbs_report.iterrows():
    if 'cores' in row[0]:
        gibbs_report.drop(index, inplace=True)
        cluster_label += 1
    else:
        cluster_labels.append(f'cluster{cluster_label}')

# Create a new column with the cluster labels
gibbs_report['cluster_label'] = cluster_labels

# Create a new column with the cluster labels
gibbs_report['cluster_label'] = cluster_labels
gibbs_report = gibbs_report.rename(columns={0:"peptide"})
desired_order = ['cluster_label', 'peptide']

# Reorder the columns
gibbs_report = gibbs_report[desired_order]


6467


In [44]:
print(gibbs_report.head())
print(len(gibbs_report))
merged_df = pd.merge(filtered_hla_a2, gibbs_report, on='peptide', how='left')
merged_df.to_csv("Y:/data/LOO_dataset.csv")


  cluster_label    peptide
1      cluster1  TICLKNEGV
2      cluster1  TTLSRHIFM
3      cluster1  ELEEICHDL
4      cluster1  MILALTVAI
5      cluster1  YMYDFILRF
6462


In [41]:
print(len(merged_df))
binders = merged_df[['cluster_label','ID', 'peptide','binder']]

8225


In [43]:
result_df = merged_df.groupby(['cluster_label'])['binder'].agg(binders='sum', non_binders=lambda x: len(x) - sum(x))
result_df.to_csv("Y:/data/LOO_dataset_binder_non_binder.csv")
print(result_df)

               binders  non_binders
cluster_label                      
cluster1           606          788
cluster2           865          960
cluster3           651         1063
cluster4           950          652
cluster5          1015          675


In [45]:
cluster_id_binder = merged_df[['cluster_label','ID','binder']]
cluster_id_binder.to_csv("Y:/data/LOO_cluster_id_binder.csv")